In [ ]:
# !pip install pyspark azure-storage-blob


In [ ]:
from azure.storage.blob import BlobServiceClient

# Azure Blob Storage connection string
azure_blob_connection_string = "cNvsvKyR7q7rnqcpX1nU32d65m71zwW4AQ6UM5jmnDem/20inTNMBW0LnGan3xV4xTqaUiNfD2Olgr8EkJ12QMmOcFh3zU2UsI6Ylocjzd2xI4+RDGr8g6WlsfviHWD+5M70kE9pC0Gn0TMt4gWhn9pWBrL+TVgXr3HdfAeikCQW2FMuFPtBWphBWgiqp6VQsW9HvObAgYrEN8x+4tFlxc1vD2JsCHn74eqhhh05vzC7Inodp6+R6uzlOqAuJV2B"

# Local file path to upload (use raw string or escape backslashes)
local_file_path = r"D:\Downloads 2\George Brown\Sem 2.5\Big data 2\Big Data project\Covid 19\COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv"

import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(local_file_path)

# Get the column names from the DataFrame
column_names = df.columns.tolist()

# Print the column names
print("Column names in the CSV file:")
print(column_names)

# Blob Storage container name and blob name
container_name = "covid19datanypd"  # Replace with your container name
blob_name = "COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv"  # Replace with the name you want to give to your blob/file

try:
    # Initialize BlobServiceClient using connection string
    blob_service_client = BlobServiceClient.from_connection_string(azure_blob_connection_string)

    # Create a blob client using the container name and blob name
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

    # Upload a file to Azure Blob Storage
    with open(local_file_path, "rb") as data:
        blob_client.upload_blob(data)

    print(f"File '{blob_name}' uploaded to Azure Blob Storage successfully.")

except Exception as ex:
    print(f"Error uploading file to Azure Blob Storage: {ex}")


Column names in the CSV file:
['date_of_interest', 'CASE_COUNT', 'PROBABLE_CASE_COUNT', 'HOSPITALIZED_COUNT', 'DEATH_COUNT', 'CASE_COUNT_7DAY_AVG', 'ALL_CASE_COUNT_7DAY_AVG', 'HOSP_COUNT_7DAY_AVG', 'DEATH_COUNT_7DAY_AVG', 'BX_CASE_COUNT', 'BX_PROBABLE_CASE_COUNT', 'BX_HOSPITALIZED_COUNT', 'BX_DEATH_COUNT', 'BX_CASE_COUNT_7DAY_AVG', 'BX_PROBABLE_CASE_COUNT_7DAY_AVG', 'BX_ALL_CASE_COUNT_7DAY_AVG', 'BX_HOSPITALIZED_COUNT_7DAY_AVG', 'BX_DEATH_COUNT_7DAY_AVG', 'BK_CASE_COUNT', 'BK_PROBABLE_CASE_COUNT', 'BK_HOSPITALIZED_COUNT', 'BK_DEATH_COUNT', 'BK_CASE_COUNT_7DAY_AVG', 'BK_PROBABLE_CASE_COUNT_7DAY_AVG', 'BK_ALL_CASE_COUNT_7DAY_AVG', 'BK_HOSPITALIZED_COUNT_7DAY_AVG', 'BK_DEATH_COUNT_7DAY_AVG', 'MN_CASE_COUNT', 'MN_PROBABLE_CASE_COUNT', 'MN_HOSPITALIZED_COUNT', 'MN_DEATH_COUNT', 'MN_CASE_COUNT_7DAY_AVG', 'MN_PROBABLE_CASE_COUNT_7DAY_AVG', 'MN_ALL_CASE_COUNT_7DAY_AVG', 'MN_HOSPITALIZED_COUNT_7DAY_AVG', 'MN_DEATH_COUNT_7DAY_AVG', 'QN_CASE_COUNT', 'QN_PROBABLE_CASE_COUNT', 'QN_HOSPITALIZED_COUN

In [2]:
import pyodbc

# Azure SQL Database connection details
server = 'blahblah.database.windows.net'
database = 'Covid19_dataNYPD'
username = 'azureuser'
password = 'Ilove85workWonder69'
driver = '{ODBC Driver 18 for SQL Server}'

# Create the connection string
conn_str = (
    f'DRIVER={driver};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    f'Encrypt=yes;'
    f'TrustServerCertificate=no;'
    f'Connection Timeout=30;'
)
# Adjust the data types according to your actual data
columns_with_types = ", ".join([f"[{col}] VARCHAR(255)" for col in column_names])

# Define the CREATE TABLE statement
table_name = "Covid19_dataNYPD"
create_table_sql = f"CREATE TABLE {table_name} ({columns_with_types});"

try:
    # Establish a connection to the Azure SQL Database
    connection = pyodbc.connect(conn_str)
    
    # Print connection details
    print("Connection established successfully!")
    print(connection.getinfo(pyodbc.SQL_DBMS_NAME))
    print(connection.getinfo(pyodbc.SQL_DBMS_VER))

    # Execute the CREATE TABLE statement
    cursor = connection.cursor()
    cursor.execute(create_table_sql)
    connection.commit()

except pyodbc.Error as ex:
    print("Error connecting to Azure SQL Database:", ex)

Connection established successfully!
Microsoft SQL Server
12.00.5564


In [3]:
def log_message(message):
    """Function to log general messages"""
    with open('process_log.txt', 'a') as f:
        f.write(message + '\n')

def log_error(error_message):
    """Function to log errors"""
    with open('error_log.txt', 'a') as f:
        f.write(error_message + '\n')

try:
    log_message("Attempting to establish connection to Azure SQL Database...")
    # Establish a connection to the Azure SQL Database
    connection = pyodbc.connect(conn_str)
    log_message("Connection established successfully!")

    # Create a cursor object
    cursor = connection.cursor()

    log_message("Reading CSV file into pandas DataFrame...")
    # Read the CSV file into a pandas DataFrame
    local_file_path = r"D:\Downloads 2\George Brown\Sem 2.5\Big data 2\Big Data project\Covid 19\COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv"
    df = pd.read_csv(local_file_path)
    log_message("CSV file read successfully!")

    # Convert data types if necessary
    df = df.astype(str)
    log_message("Converted DataFrame columns to strings.")

    # Print the DataFrame structure
    log_message(f"DataFrame structure: \n{df.info()}")

    # Use the entire DataFrame for insertion
    df_subset = df
    log_message(f"Using all rows of the DataFrame.")

    # Print the first few rows
    log_message(f"First few rows of the DataFrame subset:\n{df_subset.head()}")

    # Insert DataFrame records one by one, commit in batches
    batch_size = 1000  # Adjust the batch size as needed
    batch_values = []

    for index, row in df_subset.iterrows():
        values = tuple(row)
        batch_values.append(values)

        if (index + 1) % batch_size == 0 or (index + 1) == len(df_subset):
            try:
                insert_sql = f"INSERT INTO Covid19_dataNYPD VALUES ({', '.join('?' * len(values))})"
                cursor.executemany(insert_sql, batch_values)
                connection.commit()
                log_message(f"Committed {index + 1} rows successfully.")
                batch_values.clear()
            except pyodbc.Error as e:
                error_message = f"Error inserting batch ending at row {index + 1}: {str(e)}"
                log_error(error_message)
                log_message(error_message)
                connection.rollback()  # Rollback the batch on error

    # Final commit for any remaining rows
    if batch_values:
        try:
            cursor.executemany(insert_sql, batch_values)
            connection.commit()
            log_message(f"Committed final batch successfully.")
        except pyodbc.Error as e:
            error_message = f"Error inserting final batch: {str(e)}"
            log_error(error_message)
            log_message(error_message)
            connection.rollback()  # Rollback the final batch on error

    log_message("Data inserted successfully into the 'call_data_lapd' table.")

    # Close the cursor and connection
    cursor.close()
    connection.close()
    log_message("Connection closed successfully.")

except pyodbc.Error as ex:
    # Log the connection error to the file
    error_message = f"Error connecting to Azure SQL Database: {str(ex)}"
    log_error(error_message)
    log_message(error_message)

except Exception as ex:
    # Log any other exceptions
    error_message = f"General error: {str(ex)}"
    log_error(error_message)
    log_message(error_message)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585 entries, 0 to 1584
Data columns (total 55 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   date_of_interest                 1585 non-null   object
 1   CASE_COUNT                       1585 non-null   object
 2   PROBABLE_CASE_COUNT              1585 non-null   object
 3   HOSPITALIZED_COUNT               1585 non-null   object
 4   DEATH_COUNT                      1585 non-null   object
 5   CASE_COUNT_7DAY_AVG              1585 non-null   object
 6   ALL_CASE_COUNT_7DAY_AVG          1585 non-null   object
 7   HOSP_COUNT_7DAY_AVG              1585 non-null   object
 8   DEATH_COUNT_7DAY_AVG             1585 non-null   object
 9   BX_CASE_COUNT                    1585 non-null   object
 10  BX_PROBABLE_CASE_COUNT           1585 non-null   object
 11  BX_HOSPITALIZED_COUNT            1585 non-null   object
 12  BX_DEATH_COUNT                   1